### Load tensorflow

In [1]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [2]:
tf.enable_eager_execution()

### Collect Data

In [93]:
import pandas as pd
import numpy as np

In [110]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [111]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [113]:
data = data.drop(['date','symbol'], axis =1)

In [114]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [115]:
data = data[:1000]
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Convert Float64 to Float32

In [116]:
data.dtypes

open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [117]:
data['open'] = data['open'].astype('float32')
data['close'] = data['close'].astype('float32')
data['low'] = data['low'].astype('float32')
data['high'] = data['high'].astype('float32')
data['volume'] = data['volume'].astype('float32')

### Divide the data into train and test sets

In [118]:
from sklearn.model_selection import train_test_split
X = data.drop(['close'], axis =1)
y = data["close"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

### Normalize Train and Test Data 

In [119]:
#Normalizing the data
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [120]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [121]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [122]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [123]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [124]:
X_train

array([[1.1784944e-06, 1.1751102e-06, 1.2123350e-06, 1.0000000e+00],
       [3.2204923e-06, 3.0719973e-06, 3.2326736e-06, 1.0000000e+00],
       [1.2140832e-05, 1.2107108e-05, 1.2292594e-05, 1.0000000e+00],
       ...,
       [3.9534378e-05, 3.7634643e-05, 3.9534378e-05, 1.0000000e+00],
       [2.3261399e-04, 2.3111279e-04, 2.3355226e-04, 1.0000000e+00],
       [6.6367811e-06, 6.6367811e-06, 6.8007871e-06, 1.0000000e+00]],
      dtype=float32)

In [125]:
for i in range(100):
    
    w, b = train(X_train, np.array(y_train), w, b)
    print('Current Loss on iteration', i, loss(np.array(y_train), prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 7783.29
Current Loss on iteration 1 7447.384
Current Loss on iteration 2 7137.816
Current Loss on iteration 3 6852.516
Current Loss on iteration 4 6589.5815
Current Loss on iteration 5 6347.2676
Current Loss on iteration 6 6123.947
Current Loss on iteration 7 5918.1377
Current Loss on iteration 8 5728.46
Current Loss on iteration 9 5553.6567
Current Loss on iteration 10 5392.554
Current Loss on iteration 11 5244.0845
Current Loss on iteration 12 5107.253
Current Loss on iteration 13 4981.152
Current Loss on iteration 14 4864.935
Current Loss on iteration 15 4757.83
Current Loss on iteration 16 4659.121
Current Loss on iteration 17 4568.1523
Current Loss on iteration 18 4484.3164
Current Loss on iteration 19 4407.052
Current Loss on iteration 20 4335.844
Current Loss on iteration 21 4270.2207
Current Loss on iteration 22 4209.7397
Current Loss on iteration 23 4154.004
Current Loss on iteration 24 4102.634
Current Loss on iteration 25 4055.2927
Current Loss on

In [278]:
b.shape

TensorShape([Dimension(1)])

In [272]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

### Get the shapes and values of W and b

In [126]:
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.7464107e-03]
 [2.7223732e-03]
 [2.7687277e-03]
 [3.3516525e+01]]
Bias:
 [33.51653]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [127]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [128]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
670/670 [==============================] - 0s 183us/sample - loss: 5828.5529
Epoch 2/100
670/670 [==============================] - 0s 39us/sample - loss: 3923.9783
Epoch 3/100
670/670 [==============================] - 0s 40us/sample - loss: 3579.5837
Epoch 4/100
670/670 [==============================] - 0s 48us/sample - loss: 3517.8824
Epoch 5/100
670/670 [==============================] - 0s 51us/sample - loss: 3505.4499
Epoch 6/100
670/670 [==============================] - 0s 51us/sample - loss: 3502.9023
Epoch 7/100
670/670 [==============================] - 0s 58us/sample - loss: 3500.9678
Epoch 8/100
670/670 [==============================] - 0s 60us/sample - loss: 3501.3873
Epoch 9/100
670/670 [==============================] - 0s 63us/sample - loss: 3500.6558
Epoch 10/100
670/670 [==============================] - 0s 60us/sample - loss: 3502.3676
Epoch 11/100
670/670 [==============================] - 0s 63us/sample - loss: 3507.5586
Epoch 12/100
670/670 [=======

Epoch 93/100
670/670 [==============================] - 0s 113us/sample - loss: 3502.8810
Epoch 94/100
670/670 [==============================] - 0s 106us/sample - loss: 3503.8878
Epoch 95/100
670/670 [==============================] - 0s 115us/sample - loss: 3505.7923
Epoch 96/100
670/670 [==============================] - 0s 101us/sample - loss: 3501.1908
Epoch 97/100
670/670 [==============================] - 0s 103us/sample - loss: 3501.0615
Epoch 98/100
670/670 [==============================] - 0s 107us/sample - loss: 3502.1628
Epoch 99/100
670/670 [==============================] - 0s 103us/sample - loss: 3503.4499
Epoch 100/100
670/670 [==============================] - 0s 104us/sample - loss: 3501.4544


In [129]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [252]:
df_iris = pd.read_csv("Iris.csv")
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [253]:
X = df_iris.drop(['Id','Species'], axis =1)
y = df_iris["Species"]



### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [254]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
df_iris["Species"] = le.fit_transform(df_iris["Species"])
y=df_iris["Species"]

In [255]:
y= pd.get_dummies(y).values
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

### Divide the dataset into Training and test (70:30)

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [257]:
X_train

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
4,5.0,3.6,1.4,0.2
123,6.3,2.7,4.9,1.8
90,5.5,2.6,4.4,1.2
3,4.6,3.1,1.5,0.2
17,5.1,3.5,1.4,0.3
41,4.5,2.3,1.3,0.3
9,4.9,3.1,1.5,0.1
75,6.6,3.0,4.4,1.4
111,6.4,2.7,5.3,1.9
80,5.5,2.4,3.8,1.1


In [258]:
X_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
47,4.6,3.2,1.4,0.2
109,7.2,3.6,6.1,2.5
61,5.9,3.0,4.2,1.5
120,6.9,3.2,5.7,2.3
124,6.7,3.3,5.7,2.1
46,5.1,3.8,1.6,0.2
51,6.4,3.2,4.5,1.5
103,6.3,2.9,5.6,1.8
37,4.9,3.1,1.5,0.1
149,5.9,3.0,5.1,1.8


In [259]:
y_test.shape

(45, 3)

In [260]:
y_train

array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0,

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [261]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(10, activation='softmax'))



In [262]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [263]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [264]:

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [265]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_v1_8 (Ba (None, 4)                 16        
_________________________________________________________________
dense_29 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_30 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_31 (Dense)             (None, 3)                 27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


### Fitting the model and predicting 

In [266]:
model.fit(X_train,y_train,          
          validation_data=(X_train,y_train),
          epochs=15)

Train on 105 samples, validate on 105 samples
Epoch 1/15
105/105 [==============================] - 1s 5ms/sample - loss: 1.1849 - acc: 0.3429 - val_loss: 1.1860 - val_acc: 0.3429
Epoch 2/15
105/105 [==============================] - 0s 152us/sample - loss: 1.1787 - acc: 0.3429 - val_loss: 1.1826 - val_acc: 0.3429
Epoch 3/15
105/105 [==============================] - 0s 142us/sample - loss: 1.1746 - acc: 0.3429 - val_loss: 1.1761 - val_acc: 0.3429
Epoch 4/15
105/105 [==============================] - 0s 171us/sample - loss: 1.1683 - acc: 0.3429 - val_loss: 1.1718 - val_acc: 0.3429
Epoch 5/15
105/105 [==============================] - 0s 180us/sample - loss: 1.1636 - acc: 0.3429 - val_loss: 1.1671 - val_acc: 0.3429
Epoch 6/15
105/105 [==============================] - 0s 161us/sample - loss: 1.1590 - acc: 0.3429 - val_loss: 1.1620 - val_acc: 0.3429
Epoch 7/15
105/105 [==============================] - 0s 161us/sample - loss: 1.1540 - acc: 0.3429 - val_loss: 1.1579 - val_acc: 0.3429
Epoc

### Report Accuracy of the predicted values

In [267]:
pred = model.predict(X_test)
pred

array([[0.24053654, 0.39096498, 0.36849847],
       [0.23694304, 0.4098933 , 0.35316372],
       [0.23860824, 0.40435398, 0.35703778],
       [0.23719049, 0.40908042, 0.35372904],
       [0.23742896, 0.40849152, 0.3540795 ],
       [0.2402622 , 0.38955727, 0.3701805 ],
       [0.23829056, 0.40416843, 0.35754102],
       [0.23769398, 0.40792373, 0.35438234],
       [0.2402923 , 0.38890603, 0.37080166],
       [0.23810415, 0.40750876, 0.35438704],
       [0.24034443, 0.39002037, 0.3696352 ],
       [0.23797832, 0.4084644 , 0.35355726],
       [0.23764077, 0.40766266, 0.35469654],
       [0.23785812, 0.40821573, 0.35392612],
       [0.23627909, 0.41080472, 0.35291615],
       [0.24046706, 0.39151856, 0.36801434],
       [0.23928061, 0.40241098, 0.3583084 ],
       [0.24029419, 0.39206222, 0.36764356],
       [0.23878102, 0.40379536, 0.35742357],
       [0.23736307, 0.4088721 , 0.35376483],
       [0.24034356, 0.38884968, 0.37080678],
       [0.23879535, 0.40404105, 0.35716364],
       [0.

In [268]:
model.evaluate(X_test, y_test, verbose=0)

[1.109591343667772, 0.31111112]

In [269]:
from sklearn.metrics import classification_report

In [270]:
_,_,_=classification_report(y_test,pred)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets